# Block Diagram Systems

Core idea: Functional blocks can be connected together, block-diagram style, to build systems of larger complexity. The fundamental unit -- a "System" -- has a very specific contract that supports both modularity and ease of analysis.

High-level docs:
http://underactuated.csail.mit.edu/underactuated.html?chapter=systems

Details docs:
http://drake.mit.edu/doxygen_cxx/group__systems.html

Systems can:
- Have any number of vector-valued (or abstract-valued) inputs
- Have any number of vector-valued (or abstract-valued) outputs
- Have vector-valued continuous state, assuming it supplies
- Have vector-valued discrete state, with updates defined by _DoCalcVectorTimeDerivatives
- Have abstract-valued (discretely-updated) state

In [ ]:
# Simple continuous time system
# Subclasses VectorSystem, which provides
# a convenient constructor for a System
# with 1 vector-valued input and 1 vector-valued output.
class SimpleContinuousTimeSystem(VectorSystem):
    def __init__(self):
        VectorSystem.__init__(self,
            0,                           # Zero inputs.
            1)                           # One output.
        self._DeclareContinuousState(1)  # One state variable.

    # xdot(t) = -x(t) + x^3(t)
    def _DoCalcVectorTimeDerivatives(self, context, u, x, xdot):
        xdot[:] = -x + x**3

    # y(t) = x(t)
    def _DoCalcVectorOutput(self, context, u, x, y):
        y[:] = x